In [ ]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import sys
import os
import re

## 각각 폴더별로 분산 저장되어있는 파일들을 모두 꺼내 하나의 폴더로 통합한다.

In [27]:
before_path = input("대체 전 경로 입력")
dir_list = os.listdir(before_path)

replace_path = input("대체 후 경로 입력")

for i in dir_list:
    temp = os.listdir(before_path+"//"+i)
    os.rename(before_path+"//"+str(i)+"//"+temp[0],replace_path+"//"+str(i[:28])+".csv")

## 파일들을 모두 읽어 통합 데이터프레임을 만들고, 시간 변수를 다시 읽어들어 판다스 date 타입으로 바꾼다.

In [150]:
#통합된 폴더에서 파일들을 다시 읽어와 통합된 데이터프레임(merged_list)로 만든다.

merged_list = DataFrame()
for i in dir_list:
    temp = pd.read_csv(replace_path+"//"+i+".csv",sep = "|",encoding="EUC-KR",names=["date","conzone","type","speed"],index_col=False)
    merged_list = merged_list.append(temp)

# 인덱스 재배열
    
merged_list = DataFrame.reset_index(merged_list)
del merged_list["index"]

# 시간 인덱싱

if input("시간 인덱싱을 할까요?(시간 10분 소요), Yes : 1, No : else") == "1" :
    merged_list["dates"] = merged_list["date"].apply(lambda x: pd.to_datetime(x,format="%Y%m%d"))
    
if input("작업물을 저장할까요? yes : 1, no : else") == "1" :
    merged_list.to_csv("result.csv",encoding="utf-8",index_cols = None)

In [159]:
?merged_list.to_csv

In [160]:
merged_list.to_csv(replace_path+"//"+"result.csv",encoding="utf-8",index=False)

In [172]:
# 콘존만 저장되어있는 csv파일을 읽어 참조용 파일로 활용한다.

conzones = pd.read_csv("콘존.csv",encoding="EUC-KR")
merged_main = DataFrame(columns = ["date","conzone","type","speed"])
killswitch = "1"

# 콘존(0001XXXXX)형태에서 첫 네자리의 숫자를 입력한다.(경부고속도로 : 0010)
while True:
    lists_selected = Series()
    merged_selected = DataFrame(columns = ["date","conzone","type","speed"])
    # 작업 중단 여부를 결정(killswitch = 0이면 작업 중단)
    if killswitch == "1":
        conzone_id = input("원하는 콘존의 ID 첫 네자리를 입력하세요")
        for i in np.array(conzones["콘존ID"]):
            if i[0:4] == conzone_id:
                lists_selected = pd.concat([lists_selected,Series(i)])

        for j in lists_selected:
            temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
            merged_selected = merged_selected.append(temp)
        merged_main = merged_main.append(merged_kyongbu)
        merged_main = merged_main.reset_index(drop = True)
        merged_selected.to_csv(replace_path+"//"+"VDS_총통행거리_수정"+str(conzone_id+".csv"))
        killswitch = input("작업을 계속 할까요(continue : 1)")
    else:
        break

원하는 콘존의 ID 첫 네자리를 입력하세요0010
작업을 계속 할까요(continue : 1)0


In [173]:
data = merged_main

h = (1.5) * (np.sort(data["speed"])[round((0.25)*(len(data)+1))] + np.sort(data["speed"])[round((0.75)*(len(data)+1))])
lower = round(np.sort(data["speed"])[round((0.25)*(len(data)+1))] - h)
upper = round(np.sort(data["speed"])[round((0.75)*(len(data)+1))] + h)


## 데이터 분석을 위한 전처리

In [166]:
# 경부고속도로 ,서해고속도로, 호남고속도로, 중부내륙, 영동고속도로의 콘존ID들(0010 패밀리, 0250 패밀리)를 리스트로 일단 저장

lists_kyongbu = Series()
lists_westsea = Series()
lists_honam = Series()
lists_innerland = Series()
lists_Yeongdong = Series()

for i in np.array(conzones["콘존ID"]):
    if i[0:4] == "0010":
        lists_kyongbu = pd.concat([lists_kyongbu,Series(i)])
    if i[0:4] == "0150":
        lists_westsea = pd.concat([lists_westsea,Series(i)])
    if i[0:4] == "0251":
        lists_honam = pd.concat([lists_honam,Series(i)])
    if i[0:4] == "0450":
        lists_innerland = pd.concat([lists_innerland,Series(i)])
    if i[0:4] == "0500":
        lists_Yeongdong = pd.concat([lists_Yeongdong,Series(i)])

In [8]:
merged_list = pd.read_csv(replace_path+"VDS_구간통행속도_1일_1개월_통합.csv",encoding="utf-8",engine="python",index_col = 0)

In [165]:
# 앞에서 저장한 구간별 콘존ID 리스트와 매칭시켜서, (0010 패밀리, 0250 패밀리 등) 해당 콘존에 해당하는 데이터셋을 담아준다.

merged_kyongbu = DataFrame(columns = ["date","conzone","type","speed"])
merged_westsea = DataFrame(columns = ["date","conzone","type","speed"])
merged_honam = DataFrame(columns = ["date","conzone","type","speed"])
merged_innerland = DataFrame(columns = ["date","conzone","type","speed"])
merged_Yeongdong = DataFrame(columns = ["date","conzone","type","speed"])



while True:
    for j in lists_kyongbu:
        temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
        merged_kyongbu = merged_kyongbu.append(temp)
    for j in lists_westsea:
        temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
        merged_westsea = merged_westsea.append(temp)
    for j in lists_honam:
        temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
        merged_honam = merged_honam.append(temp)
    for j in lists_innerland:
        temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
        merged_innerland = merged_innerland.append(temp)
    for j in lists_Yeongdong:
        temp = merged_list.loc[np.where(np.array(merged_list["conzone"]) == j)]
        merged_Yeongdong = merged_Yeongdong.append(temp)
    break

NameError: name 'lists_westsea' is not defined

## 탐색적 데이터 분석(맛보기)

In [180]:
data = merged_main

h = (1.5) * (np.sort(data["speed"])[round((0.25)*(len(data)+1))] + np.sort(data["speed"])[round((0.75)*(len(data)+1))])
upper = round(np.sort(data["speed"])[round((0.75)*(len(data)+1))] + h)

print("속도가 0이 아닌 포인트는 몇개인가")
print((0<data["speed"]).value_counts())
print("=====================")
print("속도가 4분위를 벗어나는 이상값은 몇개인가")
print((data["speed"]<upper).value_counts())
print("=====================")
print("속도가 가장 높은 값은 어느 지점인가")
print(data.loc[np.argmax(data["speed"])])
print("=====================")
print("속도가 가장 낮은 지점은 어느 지점인가")
print(data.loc[np.argmin(data["speed"])])

속도가 0이 아닌 포인트는 몇개인가
True     314345
False     27813
Name: speed, dtype: int64
속도가 4분위를 벗어나는 이상값은 몇개인가
True    342158
Name: speed, dtype: int64
속도가 가장 높은 값은 어느 지점인가
conzone             0010CZS320
date                  20130901
dates      2013-09-01 00:00:00
speed                      170
type                         1
Name: 253560, dtype: object
속도가 가장 낮은 지점은 어느 지점인가
conzone             0010CZE010
date                  20180321
dates      2018-03-21 00:00:00
speed                        0
type                         1
Name: 443, dtype: object


In [181]:
#연도별 평균속도

merged_list.groupby(merged_list["dates"].apply(lambda x:x.year))["speed"].mean()

dates
2012    88.615773
2013    88.627797
2014    89.264458
2015    93.047259
2016    92.976788
2017    92.568525
2018    91.599242
Name: speed, dtype: float64

## R에서의 추가 분석을 위한 구간별 연도별 평균속도 데이터셋 만들기

In [15]:
merged_kyongbu = merged_kyongbu[merged_kyongbu["speed"] != 0]
merged_westsea = merged_westsea[merged_westsea["speed"] != 0]
merged_honam = merged_honam[merged_honam["speed"] != 0]
merged_innerland = merged_innerland[merged_innerland["speed"] != 0]
merged_Yeongdong = merged_Yeongdong[merged_Yeongdong["speed"] != 0]

In [19]:
merged_kyongbu["date_format"] = merged_kyongbu["date"].apply(lambda x: pd.to_datetime(str(x),format = "%Y%m%d",box=False))
merged_westsea["date_format"] = merged_westsea["date"].apply(lambda x: pd.to_datetime(str(x),format = "%Y%m%d",box=False))
merged_honam["date_format"] = merged_honam["date"].apply(lambda x: pd.to_datetime(str(x),format = "%Y%m%d",box=False))
merged_innerland["date_format"] = merged_innerland["date"].apply(lambda x: pd.to_datetime(str(x),format = "%Y%m%d",box=False))
merged_Yeongdong["date_format"] = merged_Yeongdong["date"].apply(lambda x: pd.to_datetime(str(x),format = "%Y%m%d",box=False))

## 만약의 사태에 대비하여 파일들을 임시로 저장한다.

In [29]:
merged_kyongbu.to_csv("속도데이터_경부.csv",encoding="utf-8")
merged_westsea.to_csv("속도데이터-서해안.csv",encoding="utf-8")
merged_honam.to_csv("속도데이터-호남.csv",encoding="utf-8")
merged_innerland.to_csv("속도데이터-중부내륙.csv",encoding="utf-8")
merged_Yeongdong.to_csv("속도데이터-영동.csv",encoding="utf-8")

In [4]:
merged_kyongbu = pd.read_csv("속도데이터_경부.csv",encoding="utf-8",engine="python",index_col = 0)
merged_westsea = pd.read_csv("속도데이터-서해안.csv",encoding="utf-8",engine="python",index_col = 0)
merged_honam = pd.read_csv("속도데이터-호남.csv",encoding="utf-8",engine="python",index_col = 0)
merged_innerland = pd.read_csv("속도데이터-중부내륙.csv",encoding="utf-8",engine="python",index_col = 0)
merged_Yeongdong = pd.read_csv("속도데이터-영동.csv",encoding="utf-8",engine="python",index_col = 0)

In [27]:
kyongbu_mean = merged_kyongbu.groupby(merged_kyongbu["date_format"].apply(lambda x : x[0:4]))["speed"].mean()

westsea_mean = merged_westsea.groupby(merged_westsea["date_format"].apply(lambda x : x[0:4]))["speed"].mean()

honam_mean = merged_honam.groupby(merged_honam["date_format"].apply(lambda x : x[0:4]))["speed"].mean()

innerland_mean = merged_innerland.groupby(merged_innerland["date_format"].apply(lambda x : x[0:4]))["speed"].mean()

Yeongdong_mean = merged_Yeongdong.groupby(merged_Yeongdong["date_format"].apply(lambda x : x[0:4]))["speed"].mean()

In [42]:
DataFrame([kyongbu_mean,westsea_mean,honam_mean,innerland_mean,Yeongdong_mean],index = ["경부","서해안","호남","중부내륙","영동"])

date_format,2012,2013,2014,2015,2016,2017,2018
경부,93.807838,93.561655,94.214452,94.716791,94.055922,93.706339,93.615070
서해안,98.711479,99.071360,99.889607,100.582614,99.516067,99.223373,99.283800
호남,95.026853,95.983864,95.827775,97.147585,96.420845,97.542540,96.838329
중부내륙,95.804951,95.740329,97.172796,98.095114,96.814999,96.310661,96.028651
영동,92.038565,91.678966,92.517549,91.860405,91.350546,91.210083,93.006983


## R에서 추가 분석을 위해 csv 형태로 저장한다.

In [37]:
mean_merged.to_csv("속도데이터-평균.csv",encoding="utf-8")

# 사용 완료